## Optimization exercise

**Goal: Train the 2nd order polynomial predictor using gradient descent. Optimize the stepsizes and compare against scikit-learn's implementation.**

In [17]:
# IMPORT PACKAGES

import pandas as pd
import numpy as np

In [18]:
data_path = 'data/'
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

#### Task I
Download the data from [here](https://drive.google.com/file/d/0Bz9_0VdXvv9bUUNlUTVrMF9VcVU/view?usp=sharing&resourcekey=0-16O9Fc5eaJH99-M7AHqHOg).

In [19]:
data_path = 'data/'
nb = pd.read_csv(data_path+'nba_games_2013_2015.csv', delimiter=';')
x = nb[['AST','REB','STL']]
y = nb['PTS']

In [20]:
x

,AST,REB,STL
0,41,43,14
1,23,43,8
2,20,39,7
3,19,47,6
4,21,43,4
...,...,...,...
7375,17,39,10
7376,26,40,10
7377,23,52,8
7378,23,41,11



#### TASK II
Create a function `psi(x)`, which transforms features AST (assists), REB (rebounds) and STL (steals) into 2nd order polynomial features (add each feature squared and each pair of features multiplied with every other)

**Input:** DataFrame x from above. It contains columns AST, REB, STL

**Output:** DataFrame with columns: AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB. The number of rows should be the same as Input.

In [21]:
def psi(x):
    xpoly2 = x
    ones = np.ones(len(xpoly2))

    # compute x^2
    xpower2 = x**2
    xpower2.columns = [column + '^2' for column in x.columns]
    # compute x * x reversed
    xreversed = x.loc[:,::-1]
    xtimesx = x*xreversed
    print(xreversed)
    xtimesx.columns = [column + '*rev' for column in x.columns]
    # add 1s
    xpoly2['1'] = ones
    xpoly2 = pd.concat([xpoly2,xpower2,xtimesx], axis=1)
    return xpoly2

# check scikit learn

In [22]:
xreversed = x.loc[:,::-1]
xtimesx = x*xreversed

#### Task III
Create a transformed data matrix **X**, where each **x** is mapped to psi(x).

HINT: We need to apply our function from Task II to matrix (DataFrame) x

In [23]:
X = psi(x)
X

      STL  REB  AST
0      14   43   41
1       8   43   23
2       7   39   20
3       6   47   19
4       4   43   21
...   ...  ...  ...
7375   10   39   17
7376   10   40   26
7377    8   52   23
7378   11   41   23
7379    4   44   17

[7380 rows x 3 columns]


/var/folders/q4/hl7yv5yn6g10s9npc_7kk08h0000gn/T/ipykernel_75267/2004311892.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xpoly2['1'] = ones


,AST,REB,STL,1,AST^2,REB^2,STL^2,AST*rev,REB*rev,STL*rev
0,41,43,14,1.0,1681,1849,196,1681,1849,196
1,23,43,8,1.0,529,1849,64,529,1849,64
2,20,39,7,1.0,400,1521,49,400,1521,49
3,19,47,6,1.0,361,2209,36,361,2209,36
4,21,43,4,1.0,441,1849,16,441,1849,16
...,...,...,...,...,...,...,...,...,...,...
7375,17,39,10,1.0,289,1521,100,289,1521,100
7376,26,40,10,1.0,676,1600,100,676,1600,100
7377,23,52,8,1.0,529,2704,64,529,2704,64
7378,23,41,11,1.0,529,1681,121,529,1681,121


#### Task IV
Create a function `p2(x,w)`, which outputs the value of the polynomial ata given row of `x` for given parameters `w`.

Inputs:
- x: DataFrame from above
- w: vector which represents beta coeficients for each column of X from the Task III.

Ouputs:
- Series of the same length as DataFrame x. Each value is a dot product of particular row in DataFrame and coeficients `w`.


HINT:
- length of w needs to be the same as number of columns of the dataframe that is returned from function `psi(x)`

Example Input:

`p2(x, [0.06, 0.05,0.03,0.01,0.02,0.02,0.04, 0.03,0.02,0.01])`

Example Output:

```
0       130.37
1        76.19
2        61.21
3        74.51
4        64.97
         ...  
7375     63.01
7376     79.59
7377     97.25
7378     78.85
7379     61.53
Length: 7380, dtype: float64

```

Our columns in the DataFrame **X** that is the output of `psi(x)` were in this order: `AST, REB, STL, 1, AST^2, REB^2, STL^2, ASTSTL, REBSTL, ASTREB`. If your columns are in different order the result can be different even for the **w**.

#### Task V
Create a function `Loss(x,y,w)`, which computes the squared loss of predicting **y** from **x** by `p2(x,w)` using parameters **w**. We have specified **y** as the variable PTS above. We will predict scored points (PTS) based on assists, rebounds and steals.


HINTS: 
- Output of `p2(x,w)` represents our predictions. `y_pred = p2(x,w)`
- Loss can be computed as:

```
np.mean((y_pred - y)**2)
```

#### Task VI
Code up the gradient descent. It should input **x**, target variable **y**, stepsize **alpha**, coeficients **w** and **maximum number of iterations**.

Steps:
1. transform input `x`
2. compute initial loss for given, x,y and w and append it to the empty list.
3. Inside the for loop, update each element of **w**, **w[i]**, using gradient descent.

HINT: `w[i] = w[i] - alpha * (1.0 / m) * errors_i.sum()` where `errors_i = (y_pred - y) * X.iloc[:, i]`. We are scaling the errors by multiplicating with values that are relevant for coeficients `w[i]` (column `i` of DataFrame `X`, output of `psi(x)`).

4. compute new loss using updated `w` and append to the list that we created in the step 2.
5. repeat steps 3 and 4 for max number of iterations times.

#### Task VII
Choose an arbitrary **w** and **alpha** and run gradient descent for 100 iterations. How does the loss behave? Does it converge to something?


#### Task VIII
Can you find which **alpha** from `[1, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001]` has the smallest loss after 100 iterations for a given **w**?